# IIITK.ai (Chat Application)

This notebook demonstrates how to create a simple chat application using Google's Gemini AI model with LangChain.

## Setup and Configuration
First, we'll import the required libraries and set up our configuration.

In [ ]:
!pip install langchain-google-genai langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime
import spacy

# Set your API key
os.environ["GOOGLE_API_KEY"] = "<replace this with your gemini api key>"

## Model Configuration
Now we'll configure the Gemini AI model using LangChain's components.

In [ ]:
# Initialize the LangChain Google Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=1.0,
    top_p=0.95,
    top_k=40,
    max_output_tokens=8192,
)

# Create a conversation memory
memory = ConversationBufferMemory()

# Define the conversation template
template = """You are an AI assistant for IIIT Kottayam students.
Current conversation:
{history}
Human: {input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# Create the conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

## Loading Mess Menu Context
Load the mess menu into the conversation memory.

In [ ]:
# Define mess menu
mess_menu = """
MESS MENU 

Monday

Breakfast: Vada Pav, Puttu, Channa Curry, Fried Chillies, Onions, Green Chutney, Red Powdered Chutney, Bread (Normal/Brown), Jam, Butter, Tea, Milk, Banana

Lunch: Jeera Rice, Kerala Rice, Roti, Beetroot Dry, Mulaku Kondattam, Rajma Curry, Pullisherry, Curd, Salad, Seasonal Fruit

Snacks: Raw-Banana Bajji, Bread, Jam, Butter, Tea, Milk

Dinner: Rice, Roti, Veg Kolhapuri, Cabbage Carrot Thoran, Spicy Dal Tadka, Rasam, Chips, Curd, Salad


Tuesday

Breakfast: Veg Utappam, Medu Vada, Sambhar, Coconut Chutney, Bread (Normal/Brown), Jam, Butter, Coffee, Milk

Lunch: Kerala Rice, Roti, Egg Bhurji, Rajma Curry, Tomato Dal Tadka, Chips, Curd, Salad, Rasam, Buttermilk

Snacks: Dal Vada, Bread, Jam, Butter, Tea, Milk

Dinner: Rice, Roti, Soya Chunk Curry (Small), Palak Dal Tadka, Rasam, Chips, Curd, Salad, Parippu Payasam


Wednesday

Breakfast: Idli, Masala Idli, Punugulu, Sambar, Groundnut Chutney, Tomato Chutney, Bread (Normal/Brown), Jam, Butter, Coffee, Milk

Lunch: Rice, Roti, Palak Dal Tadka, Crunchy Bhindi Fry, Rasam, Papad, Curd, Salad, Banana, Buttermilk

Snacks: Chilli Bajji, Bread, Jam, Butter, Tea, Milk

Dinner: Veg Fried Rice, Roti, Kadhai Paneer, Chilli Chicken, Masala Gravy, Onion Chilli Raita, Tang


Thursday

Breakfast: Vada Pav, Fried Chillies, Onions, Green Chutney, Red Powdered Chutney, Pongal, Sambar, Bread (Normal/Brown), Jam, Butter, Banana, Tea, Milk

Lunch: Rice, Puri, Amritsari Chole, Onion Dal Tadka, Cabbage Thoran, Salad, Curd, Sweet Lassi

Snacks: Kozhakkatta, Bread, Jam, Butter, Tea, Milk

Dinner: Rice, Roti, Egg Masala, Black Eyed Peas (Vanpayar) Curry, Dal Tadka, Papad, Rasam, Curd, Salad, Payasam


Friday

Breakfast: Idli, Masala Idli, Medu Vada, Groundnut Chutney, Tomato Chutney, Sambar, Bread (Normal/Brown), Jam, Butter, Tea, Milk

Lunch: Rice, Tomato Rice, Roti, Beans and Carrot Thoran, Chana Masala, Sambar, Salad, Curd, Chips, Buttermilk, Pineapple

Snacks: Potato Bajji, Bread, Jam, Butter, Tea, Milk

Dinner: Rice, Roti, Chicken Masala, Paneer Masala, Curd, Salad, Sharbat


Saturday

Breakfast: Upma/Vermicelli Upma, Sprouts, Groundnut Chutney, Mango Pickle, Bread (Normal/Brown), Jam, Butter, Banana, Coffee, Milk

Lunch: Rice, Roti, Kerala Rice, Ivy Gourd Fry, Onam Kootukari, Parippu Dal, Beetroot Pachadi, Mulakku Kondattam, Papad, Curd, Salad, Buttermilk

Snacks: Pazhampori/Dal Vada, Bread, Jam, Butter, Tea, Milk

Dinner: Roti, Rice, Rasam, Potato Roast, Onion Dal Tadka, Cabbage Curry, Curd, Salad, Fryums


Sunday

Breakfast: Puri Masala, Banana, Boiled Egg, Bread (Normal/Brown), Jam, Butter, Tea, Milk

Lunch: Chicken Biryani, Paneer Biryani, Veg Gravy, Chicken Gravy, Boondi Raita/Onion Chilli Raita, Papad, Salad, Sharbat

Snacks: Ela Ada, Bread, Jam, Butter, Tea, Milk

Dinner: Rice, Roti, Chole Curry, Bindi Masala Curry, Tomato Dal Tadka, Curd, Salad, Chips

"""

# Initialize the conversation with mess menu context
conversation.predict(input=f"This is the mess menu of the campus. Please use this context to answer queries about the menu: {mess_menu}")

## Interactive Chat
Get user input and receive model responses using LangChain.

In [ ]:
# Load the spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Set up text splitter for time analysis
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n", ".", ",", " "],
    length_function=len
)

def has_time_context(text):
    """Check if the input text contains time-related context using spaCy NLP model"""
    doc = nlp(text.lower())
    for ent in doc.ents:
        if ent.label_ in ["DATE", "TIME"]:
            return True
    return False

def get_time_context():
    """Get current date and time information"""
    now = datetime.now()
    return f"""Current time context:
- Date: {now.strftime('%A, %B %d, %Y')}
- Time: {now.strftime('%I:%M %p')}
- Day of week: {now.strftime('%A')}"""

def chat_loop():
    while True:
        user_input = input("Enter your query (or 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break
        
        # Check if query has time-related context
        if has_time_context(user_input):
            time_context = get_time_context()
            enhanced_input = f"{time_context}\n\nUser query: {user_input}"
        else:
            enhanced_input = user_input
        
        response = conversation.predict(input=enhanced_input)
        print("Assistant:", response)

# Start the chat loop
chat_loop()